In [9]:
import calcbench as cb
import pandas as pd
import requests

In [29]:
FISCAL_YEAR = 2021

In [101]:
def get_holdings(file_name: str):
    holdings = pd.read_csv(
        file_name, header=0
    )  # From https://www.barchart.com/etfs-funds/quotes/QQQ/constituents
    holdings = holdings[:-1]
    holdings.Symbol = holdings.Symbol.replace({"GOOGL": "GOOG", "BRK.B": "BRK"})
    holdings = holdings.rename({'Symbol' : 'ticker'}, axis=1)
    holdings = holdings.set_index("ticker")
    holdings = holdings["% Holding"].str.rstrip("%").astype("float") * .01
    holdings = holdings[holdings.index.dropna()]
    holdings = holdings.groupby('ticker').sum().sort_values(ascending=False) # remove duplicates
    return holdings

In [102]:
QQQ_holdings = get_holdings("./etf-constituents-06-02-2022 QQQ.csv")
SPY_holdings = get_holdings("./etf-constituents-06-02-2022 SPY.csv")

In [6]:
income_statement_metrics = [
    "Revenue",
    "CostOfRevenue",
    "GrossProfit",
    "SGAExpense",
    "OperatingExpenses",
    "OperatingIncome",
    "EBIT",
    "InterestExpense",
    "IncomeTaxes",
    "NetIncome",
]

In [22]:
company_identifiers = SPY_holdings.index.union(QQQ_holdings.index).unique().unique()[:5]

In [63]:
d = cb.standardized(
    metrics=income_statement_metrics,
    company_identifiers=company_identifiers,
    fiscal_year=FISCAL_YEAR,
    fiscal_period=cb.api_query_params.Period.Annual
)

In [146]:
by_fund_weight = d["value"].unstack(["metric", "fiscal_period"]).mul(SPY_holdings, axis=0)

In [166]:
totals = by_fund_weight.sum()

In [170]:
percent_of_revenue = totals / totals['Revenue']

In [172]:
percent_of_revenue[income_statement_metrics]

metric             fiscal_period
Revenue            2021-0           1.000000
CostOfRevenue      2021-0           0.577404
GrossProfit        2021-0           0.422596
SGAExpense         2021-0           0.063036
OperatingExpenses  2021-0           0.124586
OperatingIncome    2021-0           0.298011
EBIT               2021-0           0.305034
InterestExpense    2021-0           0.007854
IncomeTaxes        2021-0           0.039441
NetIncome          2021-0           0.257739
dtype: float64